# **El embudo de marketing: agrupación por semanas y meses**

Cuando los datos tienen ruido es difícil encontrar patrones. Por lo tanto, a veces tendrás que consolidarlo, por ejemplo, agrupándolo por semanas. Al agrupar los datos en una granularidad más pequeña (por ejemplo, de horas a semanas), las fluctuaciones se cancelan y hay menos ruido en los datos. Averigüemos cómo.

```python
import pandas as pd

df = pd.read_csv('datasets/funnel_cr_example.csv')
print(df)

#resultado
         date  clicks  regs  cr, %
0  2019-01-01    2000    36   1.80
1  2019-01-02   10000    26   0.26
2  2019-01-03    1000    15   1.50
3  2019-01-04    4000     4   0.10
4  2019-01-05    5000    24   0.48
5  2019-01-06    2000    66   3.30
6  2019-01-07    1000     2   0.20
7  2019-01-08    1500     2   0.13
```

Aquí está el primer algoritmo que viene a la mente:

1. Agrega la columna week al DataFrame.
2. Agrupa el DataFrame por esta columna.
3. Encuentra la tasa de conversión semanal media.


```python

# ¡es un error!

df['week'] = df['date'].dt.week
print(df.groupby('week')['cr, %'].mean())

week
36    1.091429
Name: cr, %, dtype: float64
```


Este enfoque es incorrecto. Bueno, los primeros dos pasos son correctos, pero el tercero no lo es. Así es como se debe hacer:

1. Agrega la columna week al DataFrame
2. Agrupa el DataFrame por esta columna.
3. Suma el número de impresiones, clics y registros de cada semana.
4. Vuelve a calcular la tasa de conversión semanal media.

Si solo promediamos los valores medios, perdemos información muy importante: el número de clics. Entonces estaríamos dando el mismo peso a la tasa de conversión del 2 de enero, cuando el sitio web tuvo 10 000 visitantes, y la tasa del 7 de enero, cuando solo hubo 1000.

```python
# correcto

df['week'] = df['date'].dt.week
df_ = df.groupby('week')[['regs', 'clicks']].sum()
print(df_['regs'] / df_['clicks'] * 100)

week
36    0.692
dtype: float64
```

## Ejercicios!

**Ejercicio 1**

Encuentra los valores semanales de CTR y CR.

Descarga los datos sobre las tasas de conversión diarias del archivo /datasets/funnel_daily.csv y guárdalos en la variable funnel_daily. Convierte los valores de la columna funnel_daily['date'] al formato datetime. Agrega la columna funnel_daily['week'] y busca allí el número de la semana para cada fecha.

Calcula el CTR y el CR semanales. Guarda el resultado como funnel_weekly.

In [ ]:
import pandas as pd

funnel_daily = pd.read_csv("/datasets/funnel_daily.csv")

funnel_daily["date"] = pd.to_datetime(funnel_daily["date"])  # escribe tu código aquí
funnel_daily["week"] = funnel_daily["date"].dt.week  # escribe tu código aquí

funnel_weekly = funnel_daily.groupby("week").agg(
    {"clicks": "sum", "impressions": "sum", "registrations": "sum"}
)
funnel_weekly["ctr, %"] = (funnel_weekly["clicks"] / funnel_weekly["impressions"]) * 100
funnel_weekly["cr, %"] = (
    funnel_weekly["registrations"] / funnel_weekly["clicks"]
) * 100

print(funnel_weekly)

"""
      clicks  impressions  registrations    ctr, %     cr, %
week
31       264        28351             23  0.931184  8.712121
32       503        57031             43  0.881976  8.548708
33       471        57184             39  0.823657  8.280255
34       424        49911             37  0.849512  8.726415
35       307        34956             25  0.878247  8.143322
"""

**Ejercicio 2**

Lee los datos sobre los gastos de publicidad (/datasets/ad_data_2.csv) y guárdalos en la variable ad_data. Guarda los datos sobre los registros (/datasets/site_data_2.csv) como site_data.

Utiliza el método merge()  para unir ad_data y site_data en la columna 'date'. Guarda el DataFrame resultante con embudos diarios como funnel_daily.  Calcula la conversión de impressions a clicks, guardándola en la columna ctr, %, y de clicks a registrations, guardándola en la columna cr, %. En funnel_daily crea las columnas 'week' y 'month'. Recupera los números de orden de las semanas y meses con los métodos dt.week y dt.month, respectivamente. Almacena los resultados en las columnas mencionadas anteriormente.

Guarda el embudo semanal en funnel_weekly. Agrupa los datos funnel_daily por la columna 'week' utilizando el método groupby(). Suma los datos de las columnas ['impressions', 'clicks', 'registrations'] dentro de la agrupación.

Haz de embudo mensual usando el mismo enfoque. Guárdalo como funnel_monthly.

Calcula el CTR y el CR para cada uno de los tres embudos. Imprime el embudo mensual.

In [ ]:
import pandas as pd

# Cargar los datos de gastos de publicidad y registros
ad_data = pd.read_csv("/datasets/ad_data_2.csv")
site_data = pd.read_csv("/datasets/site_data_2.csv")

# Fusionar los datos en la columna 'date' para obtener embudos diarios
funnel_daily = pd.merge(ad_data, site_data, on="date")

# Convertir la columna 'date' al formato datetime
funnel_daily["date"] = pd.to_datetime(funnel_daily["date"])

# Calcular CTR y CR diarios
funnel_daily["ctr, %"] = (funnel_daily["clicks"] / funnel_daily["impressions"]) * 100
funnel_daily["cr, %"] = (funnel_daily["registrations"] / funnel_daily["clicks"]) * 100

# Agregar columnas 'week' y 'month'
funnel_daily["week"] = funnel_daily["date"].dt.week
funnel_daily["month"] = funnel_daily["date"].dt.month

# Obtener embudo semanal
funnel_weekly = funnel_daily.groupby("week").agg(
    {"impressions": "sum", "clicks": "sum", "registrations": "sum"}
)
funnel_weekly["ctr, %"] = (funnel_weekly["clicks"] / funnel_weekly["impressions"]) * 100
funnel_weekly["cr, %"] = (
    funnel_weekly["registrations"] / funnel_weekly["clicks"]
) * 100

# Obtener embudo mensual
funnel_monthly = funnel_daily.groupby("month").agg(
    {"impressions": "sum", "clicks": "sum", "registrations": "sum"}
)
funnel_monthly["ctr, %"] = (
    funnel_monthly["clicks"] / funnel_monthly["impressions"]
) * 100
funnel_monthly["cr, %"] = (
    funnel_monthly["registrations"] / funnel_monthly["clicks"]
) * 100

# Imprimir embudo mensual
print(funnel_monthly)

"""
       impressions  clicks  registrations    ctr, %     cr, %
month
1           197929    1462            118  0.738649  8.071135
2           188815    1447            103  0.766359  7.118176
3           214317    1710            122  0.797884  7.134503
"""

## Embudos de producto simples

Los embudos de producto te muestran cómo los usuarios interactúan con tus sitios web y aplicaciones. Son similares a los embudos de marketing en el sentido de que los usuarios se mueven de una etapa a otra y tú analizas las conversiones en cada etapa.

Lo que los hace diferentes es la información que se utiliza para trazar los embudos. Por regla general, se agregan los datos de marketing. Contienen el número de impresiones, clics y registros por día. Se parecen a los resultados de una encuesta.

Pero los datos del producto suelen ser "en bruto". Cada fila de la tabla es un evento único, por ejemplo, "el usuario 42 accedió al sitio". Estos datos se parecen más a un formulario de encuesta individual.

Imagina una tienda en línea en busca de nuevos puntos de crecimiento en las ventas. El embudo consta de cuatro etapas:

- Clientes potenciales visitan el sitio web;
- Agregan productos a sus carritos;
- Realizan pedidos;
- Pagan.
- 
Has recibido registros de eventos del servidor. Tu tarea es trazar un embudo de producto.

Vamos a cargar los datos y vamos a examinarlos:

```python
events = pd.read_csv('data/metrics/pfunnel_demo.csv')
print(events.head())
    event_name           event_time       uid
0   pageview  2019-02-27 10:16:38  99702298
1   pageview  2019-01-24 07:20:10  67118564
2   pageview  2019-05-16 06:23:10  87503951
3   pageview  2019-07-23 08:29:40  64675633
4   pageview  2019-03-11 21:47:56  73978848
```

Los registros pueden diferir de un sistema a otro pero todos comparten tres columnas obligatorias: el nombre de un evento (event_name), la fecha y la hora en que ocurrió (event_time) y el identificador del usuario a quien le ocurrió ( uid).

La manera más sencilla de crear un embudo es calcular cuántas veces ocurrió cada uno de los eventos. Agrupemos los datos en un DataFrame en el campo event_name y busquemos el número de filas:

```python
events_count = events.groupby('event_name').agg({'uid': 'count'})
print(events_count)
event_name       uid
pageview     1084       
add_to_cart  1233
checkout      714
payment       253
```

El evento pageview ocurrió 1084 veces y add_to_cart 1233 veces. Pero el usuario primero tenía que acceder al sitio y luego agregar el producto a su carrito. Esto sugeriría que el número de eventos del carrito no puede ser mayor que el número de impresiones.

Entonces, ¿qué pasó? Es que un evento no es igual a un usuario. Una persona puede agregar varios artículos al carrito.

Ya que en el embudo necesitamos determinar una proporción de usuarios, no eventos, tenemos que reelaborar nuestro informe. Para encontrar el número de usuarios únicos en cada grupo, llamaremos nunique (número de elementos únicos) al agregar. Este método calcula el número de elementos únicos en un conjunto. Ordenemos los resultados en orden descendente:

```python
users_count = events.groupby('event_name').agg({'uid': 'nunique'})
print(users_count.sort_values(by = 'uid', ascending=False))
pageview     1084
add_to_cart   802
checkout      272
payment        97
```

Ahora vemos claramente cuántas personas llegaron a cada una de las etapas del embudo: el 80% de los usuarios que accedieron al sitio web agregaron un producto a sus carritos. Sin embargo, de 272 usuarios que comenzaron a realizar pedidos, solo 97 terminaron pagándolo. De hecho, esto no es tan raro, pero quizás podamos mejorar la conversión en esta etapa.

## Ejercicios!

**Ejercicio 1**

Analiza el embudo de productos de un servicio en línea que ayuda a las empresas a presentar sus informes anuales. El embudo tiene tres etapas: acceder al sitio web (pageview), enviar un mensaje a un empleado del servicio (chat_message) y pagar (success).

Guarda los registros del archivo /datasets/funnel_prod_events.csv en la variable events. Agrupa los datos en events por la columna event_name, agrega (agg({'uid': 'count'}) y encuentra el número de filas en cada grupo.

Guarda el resultado como events_count e imprime esta variable.

In [ ]:
import pandas as pd

events = pd.read_csv("/datasets/funnel_prod_events.csv")  # escribe tu código aquí
events_count = events.groupby("event_name").agg(
    {"uid": "count"}
)  # escribe tu código aquí

print(events_count)


**Ejercicio 2**

Agrupa los datos en events por el campo event_name y encuentra el número de usuarios únicos en cada grupo. Ordena los resultados en orden descendente.

Guarda el resultado final como users_count e imprime esta variable.

In [ ]:
import pandas as pd

events = pd.read_csv("/datasets/funnel_prod_events.csv")
users_count = (
    events.groupby("event_name")
    .agg({"uid": ["nunique"]})
    .sort_values(by=("uid", "nunique"), ascending=False)
)
print(users_count)

## Embudos de productos con secuencia de eventos

### Embudo de Usuarios en un Servicio en Línea

- **Objetivo del Embudo**:
  - Determinar cuántos usuarios pasan por cada etapa del embudo, desde acceder al sitio hasta realizar un pago.

- **Análisis Detallado**:
  - A veces, es necesario estudiar la transición detallada entre etapas del embudo, como la proporción de usuarios que realizan cada acción en el orden deseado.

- **Ejemplo de Uso del Método `pivot_table()`**:
  - Se utilizó para encontrar la hora del primer suceso de cada evento para cada usuario, creando una tabla con el tiempo de cada evento para cada usuario.

- **Cálculo de Usuarios por Etapa**:
  - Se determinó el número de usuarios que pasaron por cada etapa del embudo, verificando la hora de los eventos clave.

- **Condiciones para Cada Etapa**:
  - Se crearon variables para representar las condiciones de transición entre cada etapa del embudo.

- **Resultados**:
  - Se obtuvieron el número de visitantes, usuarios que agregaron productos al carrito, iniciaron pagos y completaron pagos.

- **Comparación con Embudo Simple**:
  - Se compararon los resultados del embudo basado en la secuencia de eventos con un embudo simple, mostrando diferencias significativas en las etapas intermedias del embudo.

- **Conclusiones**:
  - Los embudos que consideran la secuencia de eventos son más útiles para analizar patrones y generar hipótesis futuras, proporcionando datos valiosos para la toma de decisiones.


```python
import pandas as pd

# Leer datos y crear tabla pivot
events = pd.read_csv('/datasets/funnel_prod_events.csv')
users = events.pivot_table(
    index='uid', 
    columns='event_name', 
    values='event_time',
    aggfunc='min')

# Definir condiciones para cada etapa
step_1 = ~users['pageview'].isna()
step_2 = step_1 & (users['add_to_cart'] > users['pageview'])
step_3 = step_2 & (users['checkout'] > users['add_to_cart'])
step_4 = step_3 & (users['payment'] > users['checkout'])

# Calcular usuarios por etapa y mostrar resultados
n_pageview = users[step_1].shape[0]
n_add_to_cart = users[step_2].shape[0]
n_checkout = users[step_3].shape[0]
n_payment = users[step_4].shape[0]

print('Visitantes:', n_pageview)
print('Se agregó un producto al carrito:', n_add_to_cart)
print('Pago iniciado:', n_checkout)
print('Pagado:', n_payment)
```


## Ejercicios!

**Ejercicio 1**

Analiza el embudo de una tienda en línea.

Guarda los registros del archivo /datasets/events.csv como events. Imprime las primeras cinco filas de esta variable para obtener los nombres de las columnas. Para cada usuario busca el momento más temprano en que ocurrió cada evento. Guarda el resultado en la variable users e imprime también las primeras cinco filas de este DataFrame.

In [ ]:
import pandas as pd

# Cargar los registros de eventos y mostrar las primeras filas
events = pd.read_csv("/datasets/events.csv")

# Encontrar el momento más temprano de cada evento por usuario
users = events.pivot_table(
    index="uid", columns="event_name", values="event_time", aggfunc="min"
)

# Mostrar las primeras filas del DataFrame de usuarios
print(users.head(5))

**Ejercicio 2**

Encuentra el número de visitas al sitio web. Guarda el resultado como pageview_count e imprime esta variable.

In [ ]:
import pandas as pd

events = pd.read_csv("/datasets/events.csv")

users = events.pivot_table(
    index="uid", columns="event_name", values="event_time", aggfunc="min"
)

pageview_count = pageview_count = users[~users["pageview"].isna()].shape[
    0
]  # escribe tu código aquí
print("Visitantes:", pageview_count)

**Ejercicio 3**


Traza el embudo. Busca:

- el número de visitantes que agregaron productos a sus carritos después de su primera visita al sitio;
- el número de visitantes que iniciaron el proceso de pago después de agregar productos a sus carritos;
- el número de visitantes que pagaron sus pedidos después de finalizar la compra.

Guarda los resultados en las variables n_add_to_cart, n_checkout y n_payment, respectivamente.

In [ ]:
import pandas as pd

# Cargar los registros de eventos
events = pd.read_csv("/datasets/events.csv")

# Encontrar el momento más temprano de cada evento por usuario
users = events.pivot_table(
    index="uid", columns="event_name", values="event_time", aggfunc="min"
)

# Definir las condiciones para cada etapa del embudo
step_1 = ~users["pageview"].isna()
step_2 = step_1 & (users["add_to_cart"] > users["pageview"])
step_3 = step_2 & (users["checkout"] > users["add_to_cart"])
step_4 = step_3 & (users["payment"] > users["checkout"])

# Contar el número de usuarios en cada etapa del embudo
n_pageview = users[step_1].shape[0]
n_add_to_cart = users[step_2].shape[0]
n_checkout = users[step_3].shape[0]
n_payment = users[step_4].shape[0]

# Imprimir los resultados
print("Visitantes:", n_pageview)
print("Se agregó un producto al carrito:", n_add_to_cart)
print("Pago iniciado:", n_checkout)
print("Pagado:", n_payment)